IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\FAULT_M1_10


In [3]:
guasto = 2

VOLO M1 FAULT 10%

In [4]:
rcou_m1_fault10 = pd.read_csv("RCOU.csv")
rcou_m1_fault10 = rcou_m1_fault10.astype("float64")
rcou_m1_fault10 = rcou_m1_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1_fault10 = rcou_m1_fault10[((rcou_m1_fault10['C9'] >= potenza) & (rcou_m1_fault10['C10'] >= potenza) & (rcou_m1_fault10['C11'] >= potenza) & (rcou_m1_fault10['C12'] >= potenza) & (rcou_m1_fault10['C13'] >= potenza) & (rcou_m1_fault10['C14']>= potenza))]

rcou_m1_fault10 = rcou_m1_fault10.reset_index(drop=True)

pd.set_option('display.float_format', lambda x: '%.1f' % x)

display(rcou_m1_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,1648547241.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0
1,1648558959.0,1351.0,1504.0,1451.0,1414.0,1366.0,1493.0
2,1648579931.0,1467.0,1420.0,1429.0,1459.0,1448.0,1441.0
3,1648600801.0,1525.0,1374.0,1393.0,1511.0,1466.0,1443.0
4,1648611573.0,1436.0,1483.0,1442.0,1478.0,1413.0,1504.0
...,...,...,...,...,...,...,...
7959,1749459761.0,1576.0,1423.0,1531.0,1477.0,1501.0,1508.0
7960,1749470298.0,1555.0,1447.0,1515.0,1492.0,1459.0,1545.0
7961,1749480739.0,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0
7962,1749491176.0,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0


In [5]:
min=rcou_m1_fault10['TimeUS'][0]
max=rcou_m1_fault10['TimeUS'][len(rcou_m1_fault10)-1]
print(max)
print(min)

1749502709.0
1648547241.0


In [6]:
xkf1_m1_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m1_fault10 = xkf1_m1_fault10.astype("float64")
xkf1_m1_fault10 = xkf1_m1_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1_fault10 = xkf1_m1_fault10[((xkf1_m1_fault10['TimeUS'] >= min) & (xkf1_m1_fault10['TimeUS'] <= max))]
xkf1_m1_fault10 = xkf1_m1_fault10.reset_index(drop=True)
print(xkf1_m1_fault10)

           TimeUS  Roll  Pitch  Yaw
0    1648557084.0  -1.2    1.6 20.0
1    1648567530.0  -1.3    1.6 20.0
2    1648577440.0  -1.2    1.6 19.9
3    1648588414.0  -1.1    1.6 19.9
4    1648598616.0  -1.0    1.6 19.9
...           ...   ...    ...  ...
9157 1749457351.0  -1.5    0.1  6.2
9158 1749468272.0  -1.5    0.1  6.2
9159 1749478311.0  -1.4    0.0  6.1
9160 1749489170.0  -1.2   -0.1  6.1
9161 1749500410.0  -0.9   -0.1  6.2

[9162 rows x 4 columns]


In [7]:
att_m1_fault10 = pd.read_csv("ATT.csv")
att_m1_fault10 = att_m1_fault10.astype("float64")
att_m1_fault10 = att_m1_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1_fault10 = att_m1_fault10[((att_m1_fault10['TimeUS'] >= min) & (att_m1_fault10['TimeUS'] <= max))]
att_m1_fault10=att_m1_fault10.reset_index(drop=True)
print(att_m1_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw  Yaw
0    1648558171.0     -0.1  -1.2      -0.4    1.6    19.8 20.0
1    1648568640.0     -0.1  -1.3      -0.4    1.6    19.8 20.0
2    1648579278.0     -0.0  -1.2      -0.4    1.6    19.8 19.9
3    1648589763.0     -0.0  -1.1      -0.4    1.6    19.8 19.9
4    1648600108.0     -0.0  -1.0      -0.4    1.6    19.8 19.9
...           ...      ...   ...       ...    ...     ...  ...
9158 1749459114.0      0.7  -1.5      -0.3    0.1     8.1  6.2
9159 1749469562.0      0.8  -1.5      -0.3    0.1     8.1  6.2
9160 1749479836.0      0.8  -1.4      -0.3    0.0     8.1  6.1
9161 1749490199.0      0.8  -1.2      -0.3   -0.1     8.1  6.1
9162 1749501809.0      0.7  -0.9      -0.3   -0.1     8.1  6.2

[9163 rows x 7 columns]


In [8]:
if len(att_m1_fault10)>len(xkf1_m1_fault10):
    to_add=att_m1_fault10[len(xkf1_m1_fault10):]
    att_m1_fault10=att_m1_fault10[:len(xkf1_m1_fault10)]

if len(xkf1_m1_fault10)>len(att_m1_fault10):
    to_add=xkf1_m1_fault10[len(att_m1_fault10):]

for idx,i in enumerate(att_m1_fault10['Roll']):
    if(xkf1_m1_fault10['Roll'][idx] != i):
        att_m1_fault10['Roll'][idx] = ((att_m1_fault10['Roll'][idx] + xkf1_m1_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Pitch']):
    if(xkf1_m1_fault10['Pitch'][idx] != i):
        att_m1_fault10['Pitch'][idx] = ((att_m1_fault10['Pitch'][idx] + xkf1_m1_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Yaw']):
    if(xkf1_m1_fault10['Yaw'][idx] != i):
        att_m1_fault10['Yaw'][idx] = ((att_m1_fault10['Yaw'][idx] + xkf1_m1_fault10['Yaw'][idx])/2)

att_m1_fault10=pd.concat([att_m1_fault10,to_add])

print(att_m1_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw  Yaw
0    1648558171.0     -0.1  -1.2      -0.4    1.6    19.8 20.0
1    1648568640.0     -0.1  -1.3      -0.4    1.6    19.8 20.0
2    1648579278.0     -0.0  -1.2      -0.4    1.6    19.8 19.9
3    1648589763.0     -0.0  -1.1      -0.4    1.6    19.8 19.9
4    1648600108.0     -0.0  -1.0      -0.4    1.6    19.8 19.9
...           ...      ...   ...       ...    ...     ...  ...
9158 1749459114.0      0.7  -1.5      -0.3    0.1     8.1  6.2
9159 1749469562.0      0.8  -1.5      -0.3    0.0     8.1  6.1
9160 1749479836.0      0.8  -1.3      -0.3   -0.0     8.1  6.1
9161 1749490199.0      0.8  -1.1      -0.3   -0.1     8.1  6.2
9162 1749501809.0      0.7  -0.9      -0.3   -0.1     8.1  6.2

[9163 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m1_fault10 = esc_0_m1_fault10.astype("float64")
esc_0_m1_fault10 = esc_0_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1_fault10 = esc_0_m1_fault10[((esc_0_m1_fault10['TimeUS'] >= min) & (esc_0_m1_fault10['TimeUS'] <= max))]
esc_0_m1_fault10=esc_0_m1_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m1_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m1_fault10 = esc_1_m1_fault10.astype("float64")
esc_1_m1_fault10 = esc_1_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1_fault10 = esc_1_m1_fault10[((esc_1_m1_fault10['TimeUS'] >= min) & (esc_1_m1_fault10['TimeUS'] <= max))]
esc_1_m1_fault10=esc_1_m1_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m1_fault10 = esc_2_m1_fault10.astype("float64")
esc_2_m1_fault10 = esc_2_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1_fault10 = esc_2_m1_fault10[((esc_2_m1_fault10['TimeUS'] >= min) & (esc_2_m1_fault10['TimeUS'] <= max))]
esc_2_m1_fault10=esc_2_m1_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m1_fault10 = esc_3_m1_fault10.astype("float64")
esc_3_m1_fault10 = esc_3_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1_fault10 = esc_3_m1_fault10[((esc_3_m1_fault10['TimeUS'] >= min) & (esc_3_m1_fault10['TimeUS'] <= max))]
esc_3_m1_fault10=esc_3_m1_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m1_fault10 = esc_4_m1_fault10.astype("float64")
esc_4_m1_fault10 = esc_4_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1_fault10 = esc_4_m1_fault10[((esc_4_m1_fault10['TimeUS'] >= min) & (esc_4_m1_fault10['TimeUS'] <= max))]
esc_4_m1_fault10=esc_4_m1_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m1_fault10 = esc_5_m1_fault10.astype("float64")
esc_5_m1_fault10 = esc_5_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1_fault10 = esc_5_m1_fault10[((esc_5_m1_fault10['TimeUS'] >= min) & (esc_5_m1_fault10['TimeUS'] <= max))]
esc_5_m1_fault10=esc_5_m1_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m1_fault10 = imu_0_m1_fault10.astype("float64")
imu_0_m1_fault10 = imu_0_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1_fault10 = imu_0_m1_fault10[((imu_0_m1_fault10['TimeUS'] >= min) & (imu_0_m1_fault10['TimeUS'] <= max))]

imu_1_m1_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m1_fault10 = imu_1_m1_fault10.astype("float64")
imu_1_m1_fault10 = imu_1_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1_fault10 = imu_1_m1_fault10[((imu_1_m1_fault10['TimeUS'] >= min) & (imu_1_m1_fault10['TimeUS'] <= max))]

imu_2_m1_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m1_fault10 = imu_2_m1_fault10.astype("float64")
imu_2_m1_fault10 = imu_2_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1_fault10 = imu_2_m1_fault10[((imu_2_m1_fault10['TimeUS'] >= min) & (imu_2_m1_fault10['TimeUS'] <= max))]

imu_m1_fault10 = pd.concat((imu_0_m1_fault10, imu_1_m1_fault10, imu_2_m1_fault10))
imu_m1_fault10=imu_m1_fault10.groupby(imu_m1_fault10.TimeUS, as_index=False).mean()

display(imu_m1_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,1648552405.0,0.0,0.0,0.0,0.4,0.5,-9.4
1,1648558068.0,-0.2,0.0,0.0,0.2,0.5,-9.6
2,1648561764.0,0.0,0.1,0.0,0.2,0.8,-9.7
3,1648563543.0,0.2,0.1,0.0,0.4,0.6,-9.6
4,1648566595.0,0.1,-0.0,0.0,0.4,0.4,-9.5
...,...,...,...,...,...,...,...
34809,1749488276.0,0.3,-0.0,0.0,0.5,0.5,-10.3
34810,1749490094.0,0.2,-0.0,0.0,0.4,0.6,-10.4
34811,1749492809.0,0.3,-0.0,0.0,0.2,0.6,-10.6
34812,1749499200.0,0.3,-0.1,0.1,-0.1,0.6,-10.9


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m1_fault10 = pd.merge_ordered(imu_m1_fault10,att_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_0_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_1_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_2_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_3_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_4_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_5_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")
m1_fault10 = pd.merge_ordered(m1_fault10, rcou_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m1_fault10["TimeUS"] = m1_fault10["TimeUS"] - m1_fault10.iloc[0]["TimeUS"]
m1_fault10["TimeUS"] = m1_fault10["TimeUS"].astype(int)
m1_fault10["Guasto"] = guasto

In [13]:
last_cell = m1_fault10.iloc[-1, m1_fault10.columns.get_loc('TimeUS')]
print(last_cell)

100955468


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

          TimeUS
0              0
1           2857
2           5714
3           8571
4          11428
...          ...
35332  100943524
35333  100946381
35334  100949238
35335  100952095
35336  100954952

[35337 rows x 1 columns]


In [16]:
m1_fault10_final = pd.merge_ordered(m1_fault10,df)
m1_fault10_final=m1_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m1_fault10_final)

          TimeUS  GyrX  GyrY  GyrZ  AccX  AccY  AccZ  DesRoll  Roll  DesPitch  \
0              0   0.0   0.0   0.0   0.4   0.5  -9.4     -0.1  -1.2      -0.4   
1           2857   0.0   0.0   0.0   0.4   0.5  -9.4     -0.1  -1.2      -0.4   
2           5164   0.0   0.0   0.0   0.4   0.5  -9.4     -0.1  -1.2      -0.4   
3           5714   0.0   0.0   0.0   0.4   0.5  -9.4     -0.1  -1.2      -0.4   
4           8571   0.0   0.0   0.0   0.4   0.5  -9.4     -0.1  -1.2      -0.4   
...          ...   ...   ...   ...   ...   ...   ...      ...   ...       ...   
92326  100952095   0.3  -0.1   0.1  -0.1   0.6 -10.9      0.8  -1.1      -0.3   
92327  100954357   0.4  -0.1   0.1  -0.2   0.8 -11.0      0.8  -1.1      -0.3   
92328  100954568   0.4  -0.1   0.1  -0.2   0.8 -11.0      0.7  -0.9      -0.3   
92329  100954952   0.4  -0.1   0.1  -0.2   0.8 -11.0      0.7  -0.9      -0.3   
92330  100955468   0.4  -0.1   0.1  -0.2   0.8 -11.0      0.7  -0.9      -0.3   

       ...  Curr(ESC_4)  RP

In [17]:
m1_fault10_final = m1_fault10_final[m1_fault10_final.TimeUS.isin(to_be)]

In [18]:
m1_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
1,2857,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
3,5714,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
4,8571,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
7,11428,-0.2,0.0,0.0,0.2,0.5,-9.6,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92319,100943524,0.2,-0.0,0.0,0.4,0.6,-10.4,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2.0
92323,100946381,0.3,-0.0,0.0,0.2,0.6,-10.6,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
92324,100949238,0.3,-0.0,0.0,0.2,0.6,-10.6,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
92326,100952095,0.3,-0.1,0.1,-0.1,0.6,-10.9,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0


In [19]:
from datetime import timedelta
m1_fault10_final=m1_fault10_final.reset_index(drop=True)
m1_fault10_final['TimeUS'] = pd.to_datetime(m1_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
1,00:00:00.002857,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
2,00:00:00.005714,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
3,00:00:00.008571,0.0,0.0,0.0,0.4,0.5,-9.4,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
4,00:00:00.011428,-0.2,0.0,0.0,0.2,0.5,-9.6,-0.1,-1.2,-0.4,...,2.5,4410.0,3.0,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35332,00:01:40.943524,0.2,-0.0,0.0,0.4,0.6,-10.4,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2.0
35333,00:01:40.946381,0.3,-0.0,0.0,0.2,0.6,-10.6,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
35334,00:01:40.949238,0.3,-0.0,0.0,0.2,0.6,-10.6,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
35335,00:01:40.952095,0.3,-0.1,0.1,-0.1,0.6,-10.9,0.8,-1.1,-0.3,...,2.2,5600.0,3.1,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [22]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_fault10_final)/350)):
        V1=m1_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [25]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_1168\214105265.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_1168\214105265.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.0,0.0,-0.4,0.1,78.0,96.0,3.6,3.4,-0.0,...,5812.0,3.2,0.2,-1.0,3.3,120.0,119.0,15.4,8.9,2
1,1.0,-0.0,0.0,-0.4,0.1,102.0,80.0,3.6,3.4,-0.0,...,5704.5,3.2,0.2,-1.0,3.3,128.0,127.0,15.4,8.5,2
2,2.0,-0.0,0.0,-0.4,0.1,98.0,79.0,3.8,3.2,-0.0,...,5587.9,3.2,0.2,-1.0,3.3,136.0,134.0,15.5,8.2,2
3,3.0,-0.0,0.0,-0.4,0.1,102.0,71.0,4.0,3.3,-0.0,...,5462.3,3.2,0.2,-1.0,3.3,130.0,128.0,15.5,8.9,2
4,4.0,-0.0,0.0,-0.4,0.1,102.0,87.0,4.1,2.9,-0.0,...,5327.8,3.2,0.2,-1.0,3.3,138.0,135.0,15.4,9.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95.0,-0.1,0.0,-0.5,0.1,121.0,125.0,4.0,3.9,0.0,...,7576.7,3.4,0.3,-1.0,3.5,156.0,157.0,13.8,12.7,2
96,96.0,-0.1,0.0,-0.5,0.1,115.0,111.0,4.4,4.1,0.0,...,7295.0,3.4,0.3,-1.0,3.5,157.0,158.0,14.0,13.0,2
97,97.0,-0.1,0.0,-0.5,0.1,0.0,123.0,3.3,2.6,0.0,...,7003.3,3.5,0.3,-1.0,3.5,146.0,147.0,14.2,13.2,2
98,98.0,-0.1,0.0,-0.5,0.1,0.0,123.0,3.2,2.6,0.0,...,6702.8,3.5,0.3,-1.0,3.5,159.0,160.0,14.3,13.5,2


In [26]:
path_file = path_file.replace(r"M1\FAULT_M1_10", "")
os.chdir(path_file)
df_merged.to_csv('m1_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
